<a href="https://colab.research.google.com/github/mukkatharun/DeepLearningAssignments/blob/main/Assignment4/Keras_WeightsAndBiases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install wandb --upgrade

# workaround to fetch MNIST data
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

In [11]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

**Define the Sweep**

In [12]:
sweep_config = {
    'method': 'random'
    }

In [13]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [14]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

In [15]:
parameters_dict.update({
    'epochs': {
        'value': 1}
    })

In [16]:
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

In [17]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [18]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: t8yrorws
Sweep URL: https://wandb.ai/mukkatharun/pytorch-sweeps-demo/sweeps/t8yrorws


In [19]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})           

In [20]:
def build_dataset(batch_size):
   
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])
    # download MNIST training dataset
    dataset = datasets.MNIST(".", train=True, download=True,
                             transform=transform)
    sub_dataset = torch.utils.data.Subset(
        dataset, indices=range(0, len(dataset), 5))
    loader = torch.utils.data.DataLoader(sub_dataset, batch_size=batch_size)

    return loader


def build_network(fc_layer_size, dropout):
    network = nn.Sequential(  # fully-connected, single hidden layer
        nn.Flatten(),
        nn.Linear(784, fc_layer_size), nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(fc_layer_size, 10),
        nn.LogSoftmax(dim=1))

    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = F.nll_loss(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)

In [21]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: 5srph5sq with config:
wandb: 	batch_size: 40
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.09084771969769584
wandb: 	optimizer: sgd
wandb: Currently logged in as: mukkatharun (use `wandb login --relogin` to force relogin)


batch loss,▄▂▃▂█▄▁▄▆▄▃▆▂▆▂▅▅▃▂▄▃▂▁▁▂▃▆▄▂▂▃▃▁▃▄▃▁▂▁▃
epoch,▁
loss,▁
batch loss,1.44712
epoch,0
loss,1.41272


wandb: Agent Starting Run: 3flcihd7 with config:
wandb: 	batch_size: 80
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.0673593285815964
wandb: 	optimizer: adam


batch loss,▅█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,1.61253
epoch,0
loss,2.90615


wandb: Agent Starting Run: ca8ql73h with config:
wandb: 	batch_size: 88
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.08675916230817078
wandb: 	optimizer: sgd


batch loss,█▇▃▄▃▃▂▂▂▂▁▁▂▂▂▃▃▂▃▃▂▄▃▂▂▄▂▃▂▂▁▂▂▂▂▂▂▂▁▁
epoch,▁
loss,▁
batch loss,0.64971
epoch,0
loss,0.72309


wandb: Agent Starting Run: a2418o2j with config:
wandb: 	batch_size: 88
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.08070874954790529
wandb: 	optimizer: sgd


batch loss,█▆▃▂▃▃▃▂▂▃▂▂▁▃▁▄▃▂▃▃▃▃▁▃▂▂▂▂▂▂▂▃▂▂▁▂▂▁▁▁
epoch,▁
loss,▁
batch loss,0.54007
epoch,0
loss,0.68315


wandb: Agent Starting Run: td2vh3es with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.04771533424272863
wandb: 	optimizer: adam


batch loss,▇█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,2.44343
epoch,0
loss,3.85072


In [22]:
!pip install wandb -qqq
import wandb

In [23]:
import random

# Launch 5 simulated experiments
for run in range(5):
  # 1️⃣ Start a new run to track this script
  wandb.init(
      # Set entity to specify your username or team name
      # ex: entity="carey",
      # Set the project where this run will be logged
      project="basic-intro", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",})
  
  # This simple block simulates a training loop logging metrics
  offset = random.random() / 5
  for ii in range(2, 10):
      acc = 1 - 2 ** -ii - random.random() / ii - offset
      loss = 2 ** -ii + random.random() / ii + offset
      # 2️⃣ Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})
      
  # Mark the run as finished
  wandb.finish()

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▁▂▆▆█▆▆█
loss,▅█▄▄▂▁▁▂
acc,0.93736
loss,0.1026


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▁▆█▇█▇▇█
loss,█▃▄▂▁▁▁▂
acc,0.89003
loss,0.11611


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▁▅▇▅▇▇██
loss,█▅▃▃▂▁▁▁
acc,0.95535
loss,0.05122


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▁▃▅▆▆▇▇█
loss,█▆▃▂▂▂▂▁
acc,0.91077
loss,0.06584


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▃▁▂▅█▆▇█
loss,█▇▂▂▂▁▁▁
acc,0.81806
loss,0.19139


In [24]:
%wandb stacey/deep-drive/runs/1wyssjcx -h 720

In [25]:
import random

import numpy as np
import tensorflow as tf
from wandb.keras import WandbCallback

# Simple Keras Model

# Launch 20 experiments, trying different dropout rates
for run in range(20):
  # Start a run, tracking hyperparameters
  wandb.init(
      project="keras-intro",
      # Set entity to specify your username or team name
      # ex: entity="wandb",
      config={
          "layer_1": 512,
          "activation_1": "relu",
          "dropout": random.uniform(0.01, 0.80),
          "layer_2": 10,
          "activation_2": "softmax",
          "optimizer": "sgd",
          "loss": "sparse_categorical_crossentropy",
          "metric": "accuracy",
          "epoch": 6,
          "batch_size": 256
      })
  config = wandb.config

  # Get the data
  mnist = tf.keras.datasets.mnist
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train, x_test = x_train / 255.0, x_test / 255.0
  x_train, y_train = x_train[::5], y_train[::5]  # Subset data for a faster demo
  x_test, y_test = x_test[::20], y_test[::20]
  labels = [str(digit) for digit in range(np.max(y_train) + 1)]

  # Build a model
  model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(config.layer_1, activation=config.activation_1),
      tf.keras.layers.Dropout(config.dropout),
      tf.keras.layers.Dense(config.layer_2, activation=config.activation_2)
      ])

  model.compile(optimizer=config.optimizer,
                loss=config.loss,
                metrics=[config.metric]
                )

  # WandbCallback auto-saves all metrics from model.fit(), plus predictions on validation_data
  logging_callback = WandbCallback(log_evaluation=True)

  history = model.fit(x=x_train, y=y_train,
                      epochs=config.epoch,
                      batch_size=config.batch_size,
                      validation_data=(x_test, y_test),
                      callbacks=[logging_callback]
                      )

  # Mark the run as finished
  wandb.finish()

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/6
188/188 [==============================] - 2s 9ms/step - loss: 0.5040 - accuracy: 0.8516 - val_loss: 0.2990 - val_accuracy: 0.9140 - _timestamp: 1649997253.0000 - _runtime: 23.0000
Epoch 2/6
188/188 [==============================] - 1s 8ms/step - loss: 0.2443 - accuracy: 0.9276 - val_loss: 0.2358 - val_accuracy: 0.9240 - _timestamp: 1649997254.0000 - _runtime: 24.0000
Epoch 3/6
188/188 [==============================] - 1s 7ms/step - loss: 0.1750 - accuracy: 0.9513 - val_loss: 0.2035 - val_accuracy: 0.9340 - _timestamp: 1649997256.0000 - _runtime: 26.0000
Epoch 4/6
188/188 [==============================] - 1s 7ms/step - loss: 0.1456 - accuracy: 0.9588 - val_loss: 0.2115 - val_accuracy: 0.9300 - _timestamp: 1649997257.0000 - _runtime: 27.0000
Epoch 5/6
188/188 [==============================] - 1s 7ms/step - loss: 0.1130 - accuracy: 0.9674 - val_loss: 0.1792 - val_accuracy: 0.9360 - _timestamp: 1649997258.0000 

accuracy,▁▅▇▇██
epoch,▁▂▄▅▇█
loss,█▄▂▂▁▁
val_accuracy,▁▃▅▄▅█
val_loss,█▅▃▃▂▁
accuracy,0.97467
best_epoch,5
best_val_loss,0.16393
epoch,5
loss,0.09017
val_accuracy,0.95


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 1/6
188/188 [==============================] - 2s 8ms/step - loss: 0.5086 - accuracy: 0.8481 - val_loss: 0.3031 - val_accuracy: 0.9100 - _timestamp: 1649997274.0000 - _runtime: 8.0000
Epoch 2/6
188/188 [==============================] - 2s 8ms/step - loss: 0.2416 - accuracy: 0.9302 - val_loss: 0.2366 - val_accuracy: 0.9220 - _timestamp: 1649997275.0000 - _runtime: 9.0000
Epoch 3/6
188/188 [==============================] - 1s 8ms/step - loss: 0.1755 - accuracy: 0.9487 - val_loss: 0.1913 - val_accuracy: 0.9320 - _timestamp: 1649997277.0000 - _runtime: 11.0000
Epoch 4/6
188/188 [==============================] - 1s 8ms/step - loss: 0.1391 - accuracy: 0.9601 - val_loss: 0.1776 - val_accuracy: 0.9400 - _timestamp: 1649997278.0000 - _runtime: 12.0000
Epoch 5/6
188/188 [==============================] - 1s 8ms/step - loss: 0.1130 - accuracy: 0.9689 - val_loss: 0.1523 - val_accuracy: 0.9440 - _timestamp: 1649997279.0000 - _runtime: 13.0000
Epoch 6/6
188/188 [============================

...

Epoch 1/6
188/188 [==============================] - 2s 9ms/step - loss: 0.5097 - accuracy: 0.8492 - val_loss: 0.2751 - val_accuracy: 0.9080 - _timestamp: 1649997624.0000 - _runtime: 8.0000
Epoch 2/6
188/188 [==============================] - 2s 8ms/step - loss: 0.2398 - accuracy: 0.9301 - val_loss: 0.2278 - val_accuracy: 0.9080 - _timestamp: 1649997625.0000 - _runtime: 9.0000
Epoch 3/6
188/188 [==============================] - 1s 8ms/step - loss: 0.1758 - accuracy: 0.9500 - val_loss: 0.2123 - val_accuracy: 0.9300 - _timestamp: 1649997627.0000 - _runtime: 11.0000
Epoch 4/6
188/188 [==============================] - 1s 8ms/step - loss: 0.1413 - accuracy: 0.9588 - val_loss: 0.1735 - val_accuracy: 0.9320 - _timestamp: 1649997628.0000 - _runtime: 12.0000
Epoch 5/6
188/188 [==============================] - 1s 8ms/step - loss: 0.1125 - accuracy: 0.9675 - val_loss: 0.1774 - val_accuracy: 0.9380 - _timestamp: 1649997630.0000 - _runtime: 14.0000
Epoch 6/6
188/188 [============================

accuracy,▁▆▇▇██
epoch,▁▂▄▅▇█
loss,█▃▂▂▁▁
val_accuracy,▁▁▆▆██
val_loss,█▅▄▂▂▁
accuracy,0.9725
best_epoch,5
best_val_loss,0.15489
epoch,5
loss,0.09038
val_accuracy,0.94


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 1/6
188/188 [==============================] - 2s 9ms/step - loss: 0.5088 - accuracy: 0.8485 - val_loss: 0.2906 - val_accuracy: 0.9140 - _timestamp: 1649997644.0000 - _runtime: 8.0000
Epoch 2/6
188/188 [==============================] - 1s 8ms/step - loss: 0.2469 - accuracy: 0.9295 - val_loss: 0.2328 - val_accuracy: 0.9220 - _timestamp: 1649997645.0000 - _runtime: 9.0000
Epoch 3/6
188/188 [==============================] - 1s 8ms/step - loss: 0.1807 - accuracy: 0.9475 - val_loss: 0.2160 - val_accuracy: 0.9160 - _timestamp: 1649997647.0000 - _runtime: 11.0000
Epoch 4/6
188/188 [==============================] - 1s 8ms/step - loss: 0.1368 - accuracy: 0.9633 - val_loss: 0.1804 - val_accuracy: 0.9380 - _timestamp: 1649997648.0000 - _runtime: 12.0000
Epoch 5/6
188/188 [==============================] - 1s 8ms/step - loss: 0.1184 - accuracy: 0.9631 - val_loss: 0.1613 - val_accuracy: 0.9380 - _timestamp: 1649997650.0000 - _runtime: 14.0000
Epoch 6/6
188/188 [============================

accuracy,▁▆▇▇▇█
epoch,▁▂▄▅▇█
loss,█▄▂▂▁▁
val_accuracy,▁▃▂▇▇█
val_loss,█▅▄▂▁▁
accuracy,0.97383
best_epoch,4
best_val_loss,0.16129
epoch,5
loss,0.09199
val_accuracy,0.94
